In [ ]:
import os
from pathlib import Path
from typing import List, Dict

import pandas as pd
import simplejson as json
import numpy as np
from itertools import chain
from loguru import logger

import polars as pl
import torch

from justatom.modeling.mask import ILanguageModel

from justatom.configuring.prime import Config
from justatom.running.cluster import IBTRunner, IHFWrapperBackend
from justatom.modeling.prime import DocEmbedder
from justatom.clustering.prime import IUMAPDimReducer
from justatom.viewing.prime import PlotlyScatterChart

import altair as alt

In [ ]:
def ignite_dataset(where, mask:str = None) -> List[Dict]:
    docs = None
    with open(str(Path(where)), encoding="utf-8") as fin:
        docs = json.load(fin)
    if mask:
        return docs[mask]
    return docs

In [ ]:
docs = ignite_dataset(where=Path(os.getcwd()) / ".data" / "polaroids.ai.data.json")

In [ ]:
documents = [di['content'] for di in docs]
labels = [di["title"] for di in docs]

In [ ]:
def maybe_cuda_or_mps():
    if torch.cuda.is_available():
        return "cuda:0"
    elif torch.has_mps:
        return "mps"
    else:
        return "cpu"

In [ ]:
device = maybe_cuda_or_mps()
logger.info(f"Using device {device}")

In [ ]:
# model_name_or_path = Path(os.getcwd()) / "weights" / "polaroids.ai-bs=128-margin=0.4"
model_name_or_path = "intfloat/multilingual-e5-base"

In [ ]:
from justatom.processing.mask import IProcessor
from justatom.processing.prime import INFERProcessor, TripletProcessor
from justatom.processing import ITokenizer

In [ ]:
tokenizer = ITokenizer.from_pretrained("intfloat/multilingual-e5-base")
processor = INFERProcessor(tokenizer=tokenizer, max_seq_len=512)

In [ ]:
lm_model = ILanguageModel.load(model_name_or_path)

In [ ]:
embedder = DocEmbedder(model=lm_model, processor=processor, device=device)
backend_wrapper = IHFWrapperBackend(embedder, **Config.clustering.transformers_backend.toDict())

In [ ]:
bt_runner = IBTRunner(**Config.clustering.bertopic, model=backend_wrapper, verbose=True)

In [ ]:
embeddings = list(chain.from_iterable(embedder.encode(documents, verbose=True, batch_size=4)))
topics, probs = bt_runner.fit_transform(docs=documents)

In [ ]:
reducer = IUMAPDimReducer(**Config.clustering.umap.toDict())
points = reducer.fit_transform(embeddings)

In [ ]:
def prepare2d(docs, topics, labels, reduced_embeddings):
    assert reduced_embeddings.shape[1] == 2, f"Embeddings shape mismatch Exptected 2D, got {embeddings.shape[1]}D"
    COLS_MAPPING=dict(
        column_0="text",
        column_1="topic",
        column_2="label",
        column_3="x",
        column_4="y"
    )
    pl_view = pl.from_dicts(zip(docs, topics, labels, reduced_embeddings[:, 0], reduced_embeddings[:, 1]))
    pl_view = pl_view.rename(COLS_MAPPING)
    return pl_view

In [ ]:
pl_view = prepare2d(docs=documents, topics=topics, labels=labels, reduced_embeddings=points)

In [ ]:
from justatom.viewing.prime import PlotlyScatterChart

In [ ]:
chart = PlotlyScatterChart().view(pl_view, label_to_view="Вселенная")

In [ ]:
chart.write_image(f"default.png", engine='kaleido', scale=2)